# Table of Contents
* [1. Random Non-linear Schrödinger Equation](#1.-Random-Non-linear-Schrödinger-Equation)
* [2. Gross-Pitaevskii Equation (GPE)](#2.-Gross-Pitaevskii-Equation-%28GPE%29)
	* [2.1 Dimension](#2.1-Dimension)
	* [2.2 Dimensions](#2.2-Dimensions)
* [3. Dimer Method](#3.-Dimer-Method)


$\newcommand{\vect}[1]{\vec{#1}}
\newcommand{\uvect}[1]{\hat{#1}}
\newcommand{\abs}[1]{\lvert#1\rvert}
\newcommand{\norm}[1]{\lVert#1\rVert}
\newcommand{\I}{\mathrm{i}}
\newcommand{\ket}[1]{\left|#1\right\rangle}
\newcommand{\bra}[1]{\left\langle#1\right|}
\newcommand{\braket}[1]{\langle#1\rangle}
\newcommand{\op}[1]{\mathbf{#1}}
\newcommand{\mat}[1]{\mathbf{#1}}
\newcommand{\d}{\mathrm{d}}
\newcommand{\pdiff}[3][]{\frac{\partial^{#1} #2}{\partial {#3}^{#1}}}
\newcommand{\diff}[3][]{\frac{\d^{#1} #2}{\d {#3}^{#1}}}
\newcommand{\ddiff}[3][]{\frac{\delta^{#1} #2}{\delta {#3}^{#1}}}
\DeclareMathOperator{\erf}{erf}
\DeclareMathOperator{\order}{O}
\DeclareMathOperator{\diag}{diag}$

# 1. Random Non-linear Schrödinger Equation

Here is a small test-problem evolving a Schrödinger-like equation of the form

$$
  \I\ket{\dot{\psi}} = \mat{H}\ket{\psi}
$$

where the Hamiltonian depends non-linearly on $\ket{\psi}$.  We apply to this equation an external potential.

In [ ]:
from __future__ import division
import sys;sys.path.append('../../')

import numpy as np

from pytimeode import interfaces
from mmfutils import interface

class Problem(object):
    def __init__(self, N=5, seed=1, g=1.0):
        np.random.seed(seed)
        H = np.random.rand(N, N) + np.random.rand(N, N)*1j - 0.5 - 0.5j
        H = H + H.conj().T
        
        V = np.random.rand(N, N) + np.random.rand(N, N)*1j - 0.5 - 0.5j
        V = V + V.conj().T
        
        Es, Psi = np.linalg.eigh(H)
        H -= Es[0]*np.eye(N)
        Es -= Es[0]
        psi0 = Psi[:, 0]
        H -= g*np.diag(abs(psi0)**2)
        
        self.H = H
        self.V = V
        self.g = g
        self.N = N

    def Vt(self, t):
        return 0.0*self.V
    
    def get_Hy(self, y, t):
        V = self.Vt(t) + self.g*abs(y)**2
        return (self.H + V).dot(y)

    
class State(interfaces.ArrayStateMixin):
    interface.implements(interfaces.IStateForABMEvolvers)
    
    def __init__(self, p):
        self.t = 0.0
        self.p = p
        self.data = np.empty(p.N, dtype=complex)
        
    def compute_dy(self, t, dy=None, potentials=None):
        if dy is None:
            dy = self.copy()
        dy.data[...] = self.p.get_Hy(y=self.data, t=t)
        dy *= -1j
        return dy
        
interface.verifyClass(interfaces.IStateForABMEvolvers, State)

In [ ]:
p = Problem()
y = State(p)
y.data[...] = 1.0
from pytimeode import evolvers
with y.lock:
    evolver = evolvers.EvolverABM(y=y, dt=0.01)
    evolver.evolve(200)
with y.lock:
    evolver = evolvers.EvolverABM(y=y, dt=0.01)
    evolver.evolve(2)
evolver.y

# 2. Gross-Pitaevskii Equation (GPE)

Here we briefly demonstrate using the framework to solve the Gross-Pitaevskii Equation (GPE):

$$
  \I\hbar\pdiff{}{t}\psi(x,t) 
  = \left(\frac{-\hbar^2 \nabla^2}{2m} + g\abs{\psi(x,t)}^2 + V(x,t)\right)\psi(x,t).
$$

The state here is simply a complex array representing the wavefunction $\psi(x,t)$ so we can use the ``ArrayStateMixin``.  We represent the problem on an $N$-point lattice in a periodic box of length $L$ so we can use fourier methods.  This mixin allows use to define a minimal number of methods - all we really need are the following methods and attributes:

* ``data``: This attribute needs to be initialized in the constructor and defines the array used for the state.  From this the ``ArrayStateMixin`` class provides everything else needed by the evolvers.
* ``compute_dy(t, dy=None)``: Required by the ``IStateForABMEvolvers`` interface for ABM and related evolvers.
* ``apply_exp_K`` and ``apply_exp_V``: Required by the ``IStateForSpliEvolvers``.
* ``apply(expr, **kwargs)``: Optional method required by the ``INumexpr`` interface that can speed up performance by using the ``numexpr`` library.

In addition, for utility, we define the following functions.  These actually implement aspects of the discretization and basis choice – they could be redefined in order to implement a different form of discretization (finite differences, DVR basis, expansion in some basis set etc.).

* ``braket``: Computes the dot-product.  Used for projections.  In particular, it is used to make $(\op{H} - \mu)\psi$ orthogonal to $\psi$ by subtracting an appropriate chemical potential $\mu$: This ensures unitary evolution even if evolving in imaginary time.

* ``laplacian``: Computes the Laplacian of the wavefunction $\nabla^2\psi$ or applies the exponent of this times some factor $K$:  $e^{K\nabla^2}\psi$.  This is the workhorse and uses the FFT to compute the derivatives.  Another discretization (for example, finite differences) could be used by redefining this method.

A few other methods allow you to sepcify the nature of the problem:

* ``Vt``: Allows one to specify a time-dependent external potential for stirring the system.

## 2.1 Dimension

We start with a one-dimensional problem.

In [ ]:
from __future__ import division
%pylab inline --no-import-all
import sys;sys.path.append('../../')
from pytimeode import interfaces
from mmfutils import interface

class State1D(interfaces.ArrayStateMixin):
    interface.implements([interfaces.IStateForABMEvolvers,
                          interfaces.IStateForSplitEvolvers,
                          interfaces.IStateWithNormalize])

    # FFT methods - will be changed later for 2D versions fftn
    fft = staticmethod(np.fft.fft)
    ifft = staticmethod(np.fft.ifft)

    ##############################################
    # Required by IStateForABMEvolvers
    def compute_dy(self, t, dy):
        """Return `dy/dt` at time `t` using the memory in state `dy`."""
        V = self.get_V(psi=self[...], t=t)
        psi = self[...]
        Hpsi = self.laplacian(factor=-self.hbar**2/2.0/self.m) + V * psi

        if self.subtract_mu:
            # Subtract the chemical potential - ensures that evolution
            # is unitary even with complex time.
            mu = self.braket(psi, Hpsi)/self.braket(psi, psi)
            Hpsi -= mu * psi

        dy[...] = Hpsi
            
        # Here we include the phase of -1j/self.cooling_phase.
        dy *= self._phase
        return dy
    
    ##############################################
    # Required by IStateForSplitEvolvers
    linear = False   # Problem is non-linear

    def apply_exp_K(self, dt, t=None):
        r"""Apply $e^{-i K dt}$ in place"""
        #p = params = self.get_params(state=state, t=t)
        factor = -self._phase * dt * self.hbar**2 / 2.0 / self.m
        self[...] = self.laplacian(factor=factor, exp=True)
        #if self.cooling_phase is not None:
        #    self.normalize()

    def apply_exp_V(self, dt, state, t=None):
        r"""Apply $e^{-i V dt}$ in place"""
        #p = params = self.get_params(state=state, t=t)
        factor = self._phase * dt
        V = self.get_V(psi=state[...], t=t)
        self[...] *= np.exp(factor * V)
        #if self.cooling_phase is not None:
        #    self.normalize()

    ##############################################
    # Required by IStateWithNormalize
    def normalize(self, N=None):
        """Normalize to the total particle number."""
        if N is None:
            N = self._N
        self *= np.sqrt(N / self.get_N())

    ##############################################
    # Optional special methods
    def pre_evolve_hook(self):
        """Called before evolver is run.  
        
        Can be used to ensure that the object is initialized.
        """
        self.init()
    
    ##############################################
    # Optional Discretization Helpers
    # These methods are not needed by the interface -- they could instead be put
    # directly in `compute_dy` etc. but allow one to change the method of
    # discretizing the wavefunction.
    def braket(self, a, b):
        """Return the dot product of `a` with `b`.
        
        Used internally in `compute_dy()` to make the evolution
        unitary if `subtract_mu` is set.
        """
        return a.ravel().conj().dot(b.ravel()) * self.metric
    
    def get_N(self):
        """Return the particle number of the state.
        
        This depends on the basis in the sense that the appropriate
        metric must be included.
        """
        psi = self.data    
        return (abs(psi)**2).sum() * self.metric    
    
    def laplacian(self, psi=None, factor=1.0, exp=False):
        """Return the laplacian of `f(x)` times `factor` or the exponential of this.
        
        Arguments
        ---------
        factor : float
           Additional factor (mostly used with `exp=True`)
        exp : bool
           If `True`, then compute the exponential of the laplacian.
           This is used for split evolvers.
        """
        K = self._laplacian * factor
        if exp:
            K = np.exp(K)
        if psi is None:
            psi = self[...]
        return self.ifft(K * self.fft(psi))
    
    ##############################################
    # Initialization a problem specification
    # These methods specify the nature of the problem.  They
    # are not strictly needed, but allow you to adjust which
    # problem is being studied (i.e. specifying the external potential)
    def __init__(self, N=256, L=20.0, mu=32.0, hbar=1.0, m=1.0, g=1.0, omega=1.0,
                 cooling_phase=None,  # Make this 1j for imaginary time evolution
                 subtract_mu=False,   # Remove the overall phase rotation
                 data=None,
            ):
        self.t = 0.0
        self.N = N
        self.L = L
        self.hbar = hbar
        self.m = m
        self.g = g
        self.mu = mu
        self.omega = omega
        self.cooling_phase = cooling_phase
        self.subtract_mu = subtract_mu
        dtype = float if cooling_phase == 1j else complex
        self.data = np.empty(self.N, dtype=dtype)
        if data is not None:
            self.data[...] = data
        self.init()
        
        if data is None:
            # Use TF profile as initial state
            self.data[...] = np.sqrt(self.n_TF())
            
    def init(self):
        """Initialize the state.
        
        We factor out this piece from `__init__()` so that it can be
        called by `pre_evolve_hook` to recompute any necessary arrays
        before evolution in case the user changes a parameter like
        `self.L`, or `self.g`.
        """
        dx = self.L / self.N
        self.metric = dx
        
        # Abscissa centered about zero
        self.x = np.arange(self.N)*dx - self.L/2.0
        
        # Momenta
        k = 2*np.pi * np.fft.fftfreq(self.N, d=dx)
        self._laplacian = -k**2

        # Set the time-evolution phase
        if self.cooling_phase is None:
            self._phase = -1j
        else:
            self._phase = -1j / self.cooling_phase
            if not np.iscomplex(self._phase):
                # If phase is not complex, make it real so we can evolve 
                # real states without casting them to complex types.
                self._phase = self._phase.real
        
        # Record the particle number so normalize() works
        self._N = self.get_N()
        
    def n_TF(self):
        """Return the TF density.
        
        Used to provide a default initial state.
        """
        V = self.Vt(t=0)
        n_TF = np.where(self.mu > V, 
                        (self.mu - V)/self.g,
                        0.0)
        return n_TF

    def Vt(self, t):
        """Overload this to define a time-dependent external potential."""
        return self.m/2.0 * (self.omega*self.x)**2
    
    def get_V(self, psi, t=0.0):
        """Return the effective potential."""
        return self.g * abs(psi)**2 - self.mu + self.Vt(t=t)

    ##################################################################
    # Analysis Helpers
    # These are not needed for evolution, but are useful for analysis.
    def get_energy(self, t=0.0):
        """Return the energy of the state."""
        hbar2m = self.hbar**2/2.0/self.m
        psi = self.data
        n = psi.conj() * psi
        V = self.get_V(psi, t=t)
        e = -hbar2m * psi.conj()*self.laplacian() + V*n + self.g * n**2 / 2
        E = e.sum() * self.metric
        assert np.allclose(E.imag, 0)
        return E.real

    def plot(self, t=0.0):
        from matplotlib import pyplot as plt
        psi = self.data
        n = abs(psi)**2
        phase = np.angle(psi*np.exp(1j*np.pi/2))
        plt.plot(y0.x, n)
        plt.xlabel('x')
        plt.ylabel('n')
        plt.twinx()
        plt.plot(y0.x, phase)
        plt.ylabel('phase')
        E = self.get_energy(t=t)
        N = self.get_N()
        plt.title("E={}, N={}".format(E, N))

np.all([
    interface.verifyClass(interfaces.IStateForABMEvolvers, State1D),
    interface.verifyClass(interfaces.IStateForSplitEvolvers, State1D),
    interface.verifyClass(interfaces.IStateWithNormalize, State1D)])

Here we check our implementation of the methods for the Split operator evolution by computing a numerical derivative after one step and checking that this agrees with that computed by `compute_dy()`:

In [ ]:
import pytimeode.utils.testing; reload(pytimeode.utils.testing)
from pytimeode.utils.testing import TestState
s = State1D(cooling_phase=1.0+1j)
TestState(s).check_split_operator(normalize=False)

As a test problem, we will look at a domain wall oscillating in a harmonic trap

$$
  V(x, t) = \frac{m\omega^2}{2}x^2 - \mu.
$$

The length-scales for this problem are the trap-length $a = \sqrt{\hbar/m\omega}$, the inter-particle separation $1/\abs{\psi}^2$, and the interaction length-scale $l_g = \hbar^2/mg$.  If we assume that the Thomas-Fermi approximation is valid in the center of the trap, then the central density $n_0 = \mu/g$ is set by the chemical potential.  We choose units so that $\hbar = m = \omega = 1$, set the TF radius $R_{TF} = 8a$ so that $\mu = m\omega^2 R_{TF}^2/2 = 32 m\omega^2 a^2$.  Finally, we choose $g=\hbar^2/ma$ so that $l_g = a$.

In [ ]:
import scipy.optimize
sp = scipy

def solve(self, t=0, solver=sp.optimize.newton_krylov, 
          verbose=False, callback=None):
    """Find the nearest stationary state using a non-linear solver."""
    psi0 = self.data.copy()
    
    def pack(psi):
        """Return 1D real vector x from psi."""
        return psi.view(dtype=float)

    def unpack(x):
        """Return complex state psi from x."""
        return x.view(dtype=self.dtype)

    N = self.get_N()
    
    x0 = pack(psi=psi0)
    def f(x):
        self.data[...] = unpack(x=x)
        self.normalize(N=N)
        assert np.allclose(N, self.get_N())
        if callback is not None:
            callback(self)
            
        if verbose:
            print("E={}".format(self.get_energy()))            
        dy = self.compute_dy(dy=self.copy(), t=t)
        return pack(psi=dy.data)
    
    x = solver(f, x0)
    self.data[...] = unpack(x=x)
    self.normalize(N)

State1D.solve = solve

Here we start from the Thomas Fermi approximate solution:

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

# By using cooling_phase = 1j we can work with a real state which 
# is much faster for solving
y0 = State1D(cooling_phase=1j)
y0.plot()

Now we find the ground state and a state with a domain wall using the scipy non-linear solvers.  These are not very efficient and could definitely make use of the gradient information better, but they work to high precision.

In [ ]:
from IPython.display import display, clear_output
fig = plt.gcf()

def callback(y, n=[0]):
    if n[0] % 1000 == 0:
        plt.clf()
        y.plot()
        clear_output(wait=True)
        display(fig)
    n[0] += 1

callback = None  # Just use it for debugging

y1 = y0.copy()
%time y1.solve(callback=callback)
y1.plot()
display(plt.gcf())
plt.close('all')

dw0 = y0.copy()
dw0.data[...] = np.sqrt(dw0.n_TF())*np.sign(dw0.x)
dw0.normalize(N=y0.get_N())  # Normalize to same particle number.
dw1 = dw0.copy()
%time dw1.solve(callback=callback)
plt.close('all')
#clear_output()
dw1.plot()

Now we cool with imaginary time evolution to the ground state:

In [ ]:
from IPython.display import display, clear_output
from pytimeode import evolvers
fig = plt.gcf()
y0 = State(cooling_phase=1j)

with y0.lock:
    evolver = evolvers.EvolverABM(y=y0, dt=0.0005)
    for n in xrange(12):
        evolver.evolve(1000)
        plt.clf()
        y = evolver.get_y()
        y.plot(t=evolver.t)
        clear_output(wait=True)
        display(fig)
    plt.close('all')

y0 = evolver.get_y()

The split evolver is faster but less accurate.  However, one can get close by first evolving with a few large timesteps:

In [ ]:
from IPython.display import display, clear_output
from pytimeode import evolvers
fig = plt.gcf()
y0 = State(cooling_phase=1j)

with y0.lock:
    evolver = evolvers.EvolverSplit(y=y0, dt=0.01, normalize=True)
    for n in xrange(12):
        evolver.evolve(100)
        plt.clf()
        y = evolver.get_y()
        y.plot(t=evolver.t)
        clear_output(wait=True)
        display(fig)
    plt.close('all')

y0 = evolver.get_y()

Here we imprint a phase flip and cool to the lowest energy state which is a dark soliton at rest in the core of the trap.  If we use a real state with pure imaginary time evolution, this works, but if we use a complex state, numerical noise allows the evolution the break the symmetry and the soliton - a saddle point - is bypassed, ending up again in the ground state.

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
y0 = State(cooling_phase=1j)
y0.data = np.sqrt(y0.n_TF())*np.sign(y0.x)       # Real initial state
y0.data = 0j + np.sqrt(y0.n_TF())*np.sign(y0.x)  # Complex initial state

fig = plt.gcf()
y0.cooling_phase = 1j
N = y0.get_N()
with y0.lock:
    evolver = evolvers.EvolverABM(y=y0, dt=0.0001)
    evolver = evolvers.EvolverSplit(y=y0, dt=0.001, normalize=True)
    for n in xrange(20):
        evolver.evolve(200)
        plt.clf()
        y = evolver.get_y()
        
        # Constrain solution to be real
        #y.data[...] = y.data.real
        #y.data *= np.sqrt(N/y.get_N())
        y.plot(t=evolver.t)
        clear_output(wait=True)
        display(fig)
    plt.close('all')

## 2.2 Dimensions

Here we update the previous state to work with 2D problems.  We were careful with our previous implementation so that most things will work in arbitrary dimension.  Here we must generalize the `init()` method which must now compute `x`, `y`, etc. arrays separately.  The potential function `Vt()` needs minor modification, and plotting needs updating (we now use a contour plot).

In [ ]:
class State2D(State1D):
    fft = staticmethod(np.fft.fftn)
    ifft = staticmethod(np.fft.ifftn)

    ##############################################
    # Initialization a problem specification
    # These methods specify the nature of the problem.  They
    # are not strictly needed, but allow you to adjust which
    # problem is being studied (i.e. specifying the external potential)
    def __init__(self, N=(32, 32), L=(20.0, 20.0), **kw):
        # This calls the State1D constructor explicitly
        State1D.__init__(self, N=np.asarray(N), L=np.asarray(L), **kw)
            
    def init(self):
        """Initialize the state.
        
        We factor out this piece from `__init__()` so that it can be
        called by `pre_evolve_hook` to recompute any necessary arrays
        before evolution in case the user changes a parameter like
        `self.L`, or `self.g`.
        """
        dxy = self.L / self.N
        self.metric = np.prod(dxy)
        
        # Abscissa centered about zero
        xy = [np.arange(_N, dtype=float)*_L/_N - _L/2.0 
              for _N, _L in zip(self.N, self.L)]
        
        # Momenta
        kxy =[2*np.pi * np.fft.fftfreq(_N, d=_d) 
              for _N, _d in zip(self.N, dxy)]
        
        # We now use meshgrid to broadcast these into the appropriate
        # shapes so that, for example, r2 = x**2 + y**2 is a 2D array of r**2.
        self.xy = np.meshgrid(*xy, indexing='ij', sparse=True)
        kxy = np.meshgrid(*kxy, indexing='ij', sparse=True)        

        self._laplacian = sum(-_k**2 for _k in kxy)

        # Set the time-evolution phase
        if self.cooling_phase is None:
            self._phase = -1j
        else:
            self._phase = -1j / self.cooling_phase
            if not np.iscomplex(self._phase):
                # If phase is not complex, make it real so we can evolve 
                # real states without casting them to complex types.
                self._phase = self._phase.real
        
        # Record the particle number so normalize() works
        self._N = self.get_N()

    def Vt(self, t):
        """Overload this to define a time-dependent external potential."""
        r = np.sqrt(sum(_x**2 for _x in self.xy))
        return self.m/2.0 * (self.omega*r)**2
    
    def plot(self, t=0.0):
        from matplotlib import pyplot as plt
        psi = self.data
        n = abs(psi)**2
        x, y = self.xy
        plt.countourf(x.ravel(), y.ravel(), n)
        #phase = np.angle(psi*np.exp(1j*np.pi/2))
        #plt.plot(y0.x, n)
        plt.xlabel('x')
        plt.ylabel('y')
        #plt.twinx()
        #plt.plot(y0.x, phase)
        #plt.ylabel('phase')
        E = self.get_energy(t=t)
        N = self.get_N()
        plt.title("E={}, N={}".format(E, N))

np.all([
    interface.verifyClass(interfaces.IStateForABMEvolvers, State2D),
    interface.verifyClass(interfaces.IStateForSplitEvolvers, State2D),
    interface.verifyClass(interfaces.IStateWithNormalize, State2D)])

s = State2D(cooling_phase=1.0+1j)
TestState(s).check_split_operator(normalize=False)

# 3. Dimer Method

In [ ]:
def dot(self, a):
    """Return the inner product of two states."""
    return np.dot(self.data.conj().ravel(), a.data.ravel())

def norm(self):
    """Return the norm of the state."""
    return np.sqrt(self.dot(self))

def get_force(self, t=0.0):
    """Return the force on the state at the current position."""
    # Assume for now the state is real:
    assert np.allclose(self.data.imag, 0)
    F = -1j*self.compute_dy(t=t, subtract_mu=True)  # -Hy
    assert np.allclose(F.data.imag, 0)
    return F

def proj(self, x):
    """Project x along the current directions"""
    return self * self.dot(x)/self.norm()

def proj_perp(self):
    """Project x perpendicular to the current directions"""
    return self - self.proj(x)

State.dot = dot
State.norm = norm
State.get_force = get_force
State.proj = proj
State.proj_perp = proj_perp

class DimerMode(object):
    zero_torque = 0
    find_saddle = 1
    follow_saddle = 2

class Dimer(interfaces.MultiStateMixin):
    def __init__(self, y1, y2, mode):
        self.data = dict(
            R=(y1 + y2)/2, 
            r=(y1 - y2)/2)
        self.mode = mode
    
    @property
    def R(self):
        return self.data['R']

    @R.setter
    def R(self, value):
        self.data['R'] = value

    @property
    def r(self):
        return self.data['r']

    @r.setter
    def r(self, value):
        self.data['r'] = value

    @property
    def y1(self):
        return self.R + self.r

    @property
    def y2(self):
        return self.R - self.r
        
    def set_length(self, d):
        self.r *= d/norm(self.r)
    
    def get_torque(self):
        F1 = self.y1.get_force()
        F2 = self.y2.get_force()
        self.r.proj_perp(F1 - F2)

    def compute_dy(self, t, dy=None, potentials=None, subtract_mu=True):
        if dy is None:
            dy = self.copy()
        y = self.data.copy()
        Hy = self.apply_H(psi=self.data, t=t)

        if subtract_mu:
            # Subtract the chemical potential - ensures that evolution
            # is unitary even with complex time.
            mu = self.braket(y, Hy)/self.braket(y,y)
            Hy -= mu * y
            dy.data[...] = Hy
            
        dy *= (-1j/self.cooling_phase)
        return dy

        
    
y0 = State()
y1 = State() * 1.1
d = Dimer(y0, y1, mode=DimerMode.zero_torque)
d.set_length(0.1)
norm(d.r)